In [1]:
from time import clock
from math import log, sqrt
from operator import itemgetter,mul
from string import punctuation
import re


In [2]:

corpus = {} # the main corpus
document_count = {}
regex = re.compile('[%s]' % re.escape(punctuation))

def load_file(path):
    '''Load file dump and create and search indexed data structure in this
    formart {word1:{doc_id1:count, doc_id2:count},
             word2:{doc_id1:count, doc_id2:count} }'''
    documents = 0
    try:

        dump_file = open(path)
        for doc in dump_file.read().split('<doc id="'):
            '''Read each document and get words'''
            if doc[:10]:
                curid = re.search('curid=(.*?)" title', doc).group(1)
                title = re.search('title="(.*?)">', doc).group(1)
                doc_id = curid + '-' + title        #Create a unique ID combining curid and title
                if curid in document_count:
                    continue
                documents += 1
                document_count[curid] = 0
                #continue

                doc = (regex.sub('', doc)).lower().split()
                for word in doc :
                    '''For each word in a document, count how many times it occurs'''
                    document_count[curid] += 1

                    if word in corpus:
                        if doc_id in corpus[word]:
                            corpus[word][doc_id] += 1
                        else:
                            corpus[word][doc_id] = 1
                    else:
                        corpus[word] = {doc_id: 1}

        dump_file.close()

    except (Exception) as ex:
        print(str(ex))

    return documents


In [3]:
def tf_idf(doc_id, term, frequency):
    ''' tf-idf calculation'''
    id = doc_id[:doc_id.find('-')]                                  #extract the the curid
    tf = frequency / document_count[id]
    idf = log((len(document_count) / len(corpus[term])), 10)
    return tf * idf



In [4]:

def search(phrase):
    '''Search process'''

    if len(phrase) < 2:
        '''Single term search'''
        term = phrase[0]
        if term in corpus:
            ranking = []
            for doc_id, frequency in corpus[term].items():
                '''check the documents where the phrase exist return its tf-idf'''
                ranking.append((doc_id, tf_idf(doc_id, term, frequency)))
            return sorted(ranking, key=itemgetter(1))
        else:
            return None

    else:
        docs_with_terms = [] #List of documents that contain search terms
        for term in phrase:
            if term in corpus:
                id = corpus[term].keys()
                docs_with_terms.append(set((id)))
            else:
                return None

        #Get the intersection of the documents that contain all search terms
        doc_intersection = dict.fromkeys(set.intersection(*docs_with_terms),0) 
        search_terms =set(phrase)
        for document in doc_intersection:
            for term in search_terms:
                #Add cumulative tf-idf of all the search terms for each document
                doc_intersection[document] +=tf_idf(document,term,corpus[term][document])                  
        return sorted(doc_intersection.items(), key=itemgetter(1))


In [5]:
def run_search():
    '''Excute program'''
    print('Welcome to the search engine!\n'
          'type:\n'
          '{0:<5}load <filename> : to load new documents\n'
          '{0:<5}search <words> : to search for word(s) in the loaded documents \n'
          '{0:<5}q or exit to end the program'.format(''))
    print()
    while True:
        command = input('command>').lower().strip()

        if len(command) > 0:
            if command[:4] == 'load':
                time = 0
                start = clock()
                num_of_docs = load_file(command[4:].strip())
                end = clock()
                time += end - start
                avg =(time / num_of_docs) * 1000 if num_of_docs > 0 else 0
                print('Loaded {0} documents in {1:.2f} seconds ({2:.2f} ms per document).'
                      .format(num_of_docs,time, avg))
                print()
            elif command[:6] == 'search':
                if len(corpus) < 1:
                    print('The corpus is empty. Please load documents! ')
                    continue
                else:
                    # remove punctuations from phrase
                    search_phrase = (regex.sub('', command[6:])).lower().split()  
                    timer = 0
                    start = clock()
                    search_result = search(search_phrase)
                    end = clock()
                    timer += end - start
                    print()
                    print('Search completed in {0:.2f} ms.'.format(timer * 1000))
                    if search_result:
                        print()
                        print('Titles and urls of matching documents:')
                        print('----------------------------------------')

                        for i in range(1, len(search_result) + 1):
                            id = search_result[-i][0]
                            url = '->https://en.wikipedia.org/wiki?curid=' + str(id[:id.find('-')])
                            title = id[id.find('-') + 1:]
                            print('{0:60s}{1}'.format(title, url))
                        print()
                    else:
                        print('No match for the search term. Retry!')


            elif command[0] == 'q' or command[:4] == 'exit':
                break
            else:
                print('Command not recognised. Retry!')
        else:
            continue
    return




# Testing

run_search()


Welcome to the search engine!
type:
     load <filename> : to load new documents
     search <words> : to search for word(s) in the loaded documents 
     q or exit to end the program

command>load wiki_dump1.txt
Loaded 243 documents in 0.65 seconds (2.68 ms per document).

command>search software 

Search completed in 0.17 ms.

Titles and urls of matching documents:
----------------------------------------
Dan Bricklin                                                ->https://en.wikipedia.org/wiki?curid=8668
Djbdns                                                      ->https://en.wikipedia.org/wiki?curid=8736
BIND                                                        ->https://en.wikipedia.org/wiki?curid=8735
Dia (software)                                              ->https://en.wikipedia.org/wiki?curid=9069
Dragon 32/64                                                ->https://en.wikipedia.org/wiki?curid=8650
Debian GNU/Hurd                                             ->https://en.w